In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-125233")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for your CPU cluster
from azureml.core.compute_target import ComputeTargetException
cpu_cluster_name = "cpu-cluster-mla"

   # Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                              max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

    cpu_cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os
#import numpy as np

# Specify parameter sampler
ps = RandomParameterSampling({"--C": uniform(0.2, 1),
                             "--max_iter": choice(50, 100, 200)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)


mydir = os.getcwd()
# Create a SKLearn estimator for use with train.py
est = SKLearn(mydir,compute_target=cpu_cluster, entry_script='train.py')


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=4,
                                    max_concurrent_runs=4)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hdr= exp.submit(config=hyperdrive_config,show_output=True)
RunDetails(hdr).show()

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hdr.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])



In [ ]:
os.makedirs('outputs',exist_ok=True)
joblib.dump(value=best_run.id, filename='outputs/model.joblib')

In [ ]:
best_run.register_model(model_name='best_model', model_path='outputs/model.joblib')

In [ ]:
best_run.get_file_names()[-1]

In [ ]:
# insert this after fitting the model
best_run.register_model(model_name='best_model', model_path='outputs/model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

df = TabularDatasetFactory.from_delimited_files('https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv')

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(df)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y)

In [ ]:
#bring together them again
import pandas as pd
dataset = pd.concat([x_train,y_train],axis=1)

In [ ]:
#Additional settings:
automl_settings = {
    'enable_early_stopping': True,
    'iteration_timeout_minutes':5,
    'max_concurrent_iterations':4,
    'featurization': 'auto'
}

In [ ]:

from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=dataset,
    label_column_name='y',
    n_cross_validations=5,
    **automl_settings
    )

In [ ]:
# Submit your automl run

automl_run = exp.submit(automl_config,show_output=True)

In [ ]:
# Retrieve and save your best automl model.

automl_run.get_output()

In [ ]:
best_automl_run, best_model = automl_run.get_output()
best_automl_run.register_model(model_name = "best_run_automl.pkl", model_path = './outputs/')